In [35]:
class stop:

    def __init__(self, stop_id, stop_name,
                 lat, lon, wheelchair_boarding):
        self.stop_id = stop_id
        self.stop_name = stop_name
        self.lat = lat
        self.lon = lon
        self.wheelchair_boarding = wheelchair_boarding

class trip:

    # data record time:
    # - 20200823 - 20201003 for 92, 20201004 - 20201128 for 93
    # - here we'll pick 93
    # service type included: WKD, SAT, SUN, HOL, MWK
    def __init__(self, trip_id, service_type, headsign):
        self.trip_id = trip_id
        self.service_type  = service_type  # 93_WKD, 93_SAT, etc
        self.headsign = headsign

class stop_time:

    def __init__(self, trip_id, stop_id, arrival_time,
                 departure_time, distance_travel, headsign,
                 stop_sequence):
        self.trip_id = trip_id
        self.stop_id = stop_id
        self.arrival_time = arrival_time
        self.departure_time = departure_time
        self.distance_traveled = distance_travel
        self.headsign = headsign
        self.stop_sequence = stop_sequence


class route:

    def __init__(self, route_id, route_num, service_id, service_name, suspended):
        self.route_id = route_id
        self.route_num = route_num
        self.service_id = service_id  # 92 = excluded suspended service; 93 = included
                                      # 7 & 20 are special cases (only suspended some trips)
        self.service_name = service_name
        self.suspended = suspended  # suspended = True/False due to pandemic
        self.stops = []
        self.trips_wkd = {}  # trip_id: (reference to trip, reference to stop_time)
        self.trips_sat = {}
        self.trips_sun = {}
        self.trips_hol = {}
        self.trips_mwk = {}
        self.stop_times = []

    def add_stops(self, stop):
        self.stops.append(stop)

    def add_trips(self, trip_id, trip):
        service_type = trip.service_type[3:6].lower()
        if service_type == "wkd":
            self.trips_wkd.append(trip)
        elif service_type == "sat":
            self.trips_sat.append(trip)
        elif service_type == "sun":
            self.trips_sun.append(trip)
        elif service_type == "hol":
            self.trips_hol.append(trip)
        else:
            self.trips_mwk.append(trip)

    def add_stop_times(self, stop_time):
        self.stop_times.append(stop_time)

In [15]:
import pandas as pd

In [16]:
trips_df = pd.read_csv("../../../data/mmt_gtfs/trips.csv")
len(set(trips_df["route_short_name"]))  # 47

47

In [17]:
routes_df = pd.read_csv("../../../data/mmt_gtfs/routes.csv")
len(set(routes_df["route_short_name"]))  # 63
routes_df.head()

,route_id,service_id,agency_id,route_short_name,route_long_name,route_service_name,route_desc,route_type,route_url,route_color,route_text_color,bikes_allowed
0,9027,92,MMT,1,NaN,OLD UNIV:CAP SQR,Weekday schedule trips suspended until further...,3,http://www.cityofmadison.com/metro/routes-sche...,E3D23E,000000,1
1,9028,92,MMT,2,NaN,WEST TP:NORTH TP,Daily schedule trips operate regularly every 3...,3,http://www.cityofmadison.com/metro/routes-sche...,981F66,FFFFFF,1
2,9029,92,MMT,3,NaN,WEST TP:EAST TP,Weekday schedule trips suspended until further...,3,http://www.cityofmadison.com/metro/routes-sche...,CD8382,000000,1
3,9030,92,MMT,4,NaN,SOUTH TP:NORTH TP,Daily schedule trips operate regularly every 6...,3,http://www.cityofmadison.com/metro/routes-sche...,42784D,000000,1
4,9031,92,MMT,5,NaN,SOUTH TP:EAST TP,Daily schedule trips operate regularly every 6...,3,http://www.cityofmadison.com/metro/routes-sche...,B46A5B,000000,1


In [18]:
stops_df = pd.read_csv("../../../data/bus_route_stop_info/stops.csv")
stops_df.head()

,Unnamed: 0,OBJECTID,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,agency_id,location_t,...,Light,Reaitime,PK_MD_WE,jurisdicti,relative_p,cardinal_d,primary_st,address_ra,cross_loca,geometry
0,0,12792,1110,1110,E Dayton & Wisconsin (WB),This EVENT/DETOUR ONLY stop (#1110) is westbou...,43.076428,-89.385920,MMT,0,...,0,0,0,CMAD,3,270.0,E Dayton,2,Wisconsin,POINT (-89.38591919586982 43.07643036088118)
1,1,12793,1111,1111,W Doty & M L K Junior (EB),This EVENT/DETOUR ONLY stop (#1111) is eastbou...,43.072938,-89.382354,MMT,0,...,0,0,0,CMAD,3,90.0,W Doty,1,M L K Junior,POINT (-89.38235320566406 43.07294036311185)
2,2,12794,1124,1124,S Fairchild & W Main (SB),This EVENT/DETOUR ONLY stop (#1124) is southbo...,43.072467,-89.385273,MMT,0,...,0,0,0,CMAD,3,180.0,S Fairchild,98,W Main,POINT (-89.38527220696201 43.07246935942042)
3,3,12795,1135,1135,N Webster & E Mifflin (NB),This EVENT/DETOUR ONLY stop (#1135) is northbo...,43.077272,-89.383413,MMT,0,...,0,0,0,CMAD,1,0.0,N Webster,101,E Mifflin,POINT (-89.38341219351082 43.07727436415694)
4,4,12796,1140,1140,E Washington & N Webster (WB),This EVENT/DETOUR ONLY stop (#1140) is westbou...,43.076390,-89.381930,MMT,0,...,0,0,0,CMAD,3,270.0,E Washington,200,N Webster,POINT (-89.38192919600975 43.07639236517169)


In [40]:
# storing route info to routes_dict
# route_num: {service_id: route_object}
routes_dict = {route_num: {} for route_num in set(routes_df["route_short_name"])}
for idx, row in routes_df.iterrows():
    suspended = True if "Weekday schedule trips suspended until further notice." in row["route_desc"] else False
    routes_dict[row["route_short_name"]][row["service_id"]] = route(row["route_id"],
                                                                    row["route_short_name"],
                                                                    row["service_id"],
                                                                    row["route_service_name"],
                                                                    suspended)

# test if # of total routes including suspended service == 63
# and  if # of total routes excluding suspended service == 47
suspended_cnt = 0
suspended = []
not_suspended_cnt = 0
for key, val in routes_dict.items():
    for sub_key, sub_val in val.items():
        if sub_key == 92:
            if routes_dict[key][sub_key].suspended == False:
                suspended_cnt += 1
                suspended.append(routes_dict[key][sub_key].route_num)
        if sub_key == 93:
            not_suspended_cnt += 1

suspended_cnt, not_suspended_cnt

(47, 63)

In [41]:
tr92 = trips_df[trips_df["service_id"].str.startswith("92")]
tr93 = trips_df[trips_df["service_id"].str.startswith("93")]
len(set(tr92["route_short_name"])), len(set(tr93["route_short_name"]))

(47, 47)

In [42]:
# def __init__(self, route_id, route_num, trip_id, service_type, headsign):
#         self.route_id = route_id
#         self.route_num = route_num
#         self.trip_id = trip_id
#         self.service_type  = service_type  # 93_WKD, 93_SAT, etc
#         self.headsign = headsign
for route_num in routes_dict.keys():
    trips_df2 = trips_df[trips_df["route_short_name"].isin([route_num])
                         & (trips_df["service_id"].str.startswith("93"))]

    trip_obj = 0
    stop_time_obj = 0

    # trip object
    for idx, row in trips_df2.iterrows():
        trip_obj = trip(route_id=row["route_id"],
                        route_num=row["route_short_name"],
                        trip_id=row["trip_id"],
                        service_type=row["service_id"][:6],
                        headsign=row["trip_headsign"])

        # routes_dict[route_num][93].add_trips(trip_obj)

    stop_times_df2 = stop_times_df[stop_times_df[trip_id]]

    # stop time object
    for idx, row in stop_times_df2.iterrows():



# test if these values == (72, 68, 68, 58, 68)
len(routes_dict[2][93].trips_wkd), len(routes_dict[2][93].trips_sat), len(routes_dict[2][93].trips_sun), len(routes_dict[2][93].trips_hol), len(routes_dict[2][93].trips_mwk)

(72, 68, 68, 58, 68)

In [48]:
stop_times_df = pd.read_csv("../../../data/mmt_gtfs/stop_times.csv")
stop_times_df.head()

,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled
0,9999999,1,7605,0,1,5:27:00,5:27:00,1,EAST TRANSFER,0.0249
1,9999999,2,7739,0,0,5:27:40,5:27:40,0,EAST TRANSFER,0.2003
2,9999999,3,7119,0,0,5:28:33,5:28:33,0,EAST TRANSFER,0.4352
3,9999999,4,7107,0,0,5:28:53,5:28:53,0,EAST TRANSFER,0.5255
4,9999999,5,7663,0,0,5:29:51,5:29:51,0,EAST TRANSFER,0.7820


In [61]:
# for route_num in routes_dict.keys():
#     stop_times_df2 = stop_times_df[stop_times_df["trip_id"].isin(routes_dict[route_num][93].trip)]
